##### This notebook loads the Venue_locations.csv to places of interest for the venues got from ticketmaster API
##### It parses through the json files and saves the required data in FourSq_data.csv

In [ ]:
import pandas as pd
import json
import os
import requests
from get_data_API import get_venues_fs
FourSq_key=os.getenv('FOURSQUARE_API_KEY')


In [ ]:
Lat_Lon_df=pd.read_csv('Venue_locations.csv')
Lat_Lon_df

## Calling API to get data for the parameters set.

In [ ]:
places=[]

for index, row in Lat_Lon_df.iterrows():
    
    venue=row['Venue']
    lat=row['latitude']
    lon=row['longitude']
    res_fs = get_venues_fs(venue,lat,lon,FourSq_key)
    if res_fs.status_code == 200:
        data = res_fs.json()
        if 'results' in data and len(data['results'])>0:
            places.append(data['results'])
        # dump data into json file
            with open(f'foursq.json', 'w') as f: 
                    json.dump(places, f) 
            print("Data dumped for index:", index)
        else:
            print("No businesses found in data for index:", index)
            places.append("N/A")
        
    else:
        print(f"Error: {res.status_code}, {res.text}")



## To extract required information from the json file

In [ ]:
with open(f'foursq.json','r') as f:
    places=json.load(f)

In [ ]:
#places[0]#[0]#['categories'][0]
#places[0][0]['distance']
#places[0][0]['location']#['formatted_address']
#places[20]#[0]['name']
#places[0][1]#.keys()



In [ ]:
"""
data required from the json files

categories-name
distance-
location-formatted_address,address
name

"""

## Extracting the information required from json and matching it with the venues in the data frame above 

In [ ]:
# create emply lists for 
category=[]
distance_=[]
address=[]
POI=[]
venue=[]
city=[]
state=[]
latitu



In [ ]:
# to iterate through the df, to get the places for each venue location 

for index, row in Lat_Lon_df.iterrows():
    
    Venue=row['Venue']
    City=row['City']
    State=row['State']
#to get places from json to match the index of the venue
    data=places[index]
    
    for items in data:
        if isinstance(items, dict):  # Check if item is a dictionary
            cat=[] # its nested hence we need to initialize the empty list inside
            for id in items['categories']:
            
                cat.append(id['name'])
            category.append(cat)
        
            distance_.append(items['distance'])
# Get address information
            location = items['location']
            address_info = location.get('address')
            if address_info:
                address.append(address_info)
            else:
                address.append(location.get('formatted_address', 'N/A'))
            POI.append(items['name'])
            venue.append(Venue)
            city.append(City)
            state.append(State)
        
        else:
            category.append('N/A')
            address.append('N/A')
            distance_.append('N/A')
            POI.append('N/A')
            venue.append(Venue)
            city.append(City)
            state.append(State)
            
  
  
foursq_df=pd.DataFrame({'Venue':venue,
                        'City':city,
                        'State':state,
                        'POI':POI,
                        'Category':category,
                        'Address':address,
                        'Distance From Venue':distance_
                       })    
        
foursq_df.head()   
    
    

In [ ]:
# Convert lists to strings
foursq_df['Category'] = foursq_df['Category'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)
foursq_df.head()

In [ ]:
# Drop duplicates based on entire rows of data
foursq_df.drop_duplicates(inplace=True)
foursq_df.shape

In [ ]:
foursq_df.to_csv('FourSq_data.csv',index=False)